In [ ]:
# don't run locally
!git clone https://github.com/char-tan/mode_connectivity.git

In [ ]:
!cp mode_connectivity/train/mlp_mnist_train.py mode_connectivity/

In [ ]:
!python mode_connectivity/mlp_mnist_train.py --lr 0.001
# I just picked a random learning rate